In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
movies = pd.read_csv('movies.dat',sep='::',header=None,engine='python',encoding = 'latin-1')

In [3]:
print(movies)

         0                                                  1  \
0        1                                   Toy Story (1995)   
1        2                                     Jumanji (1995)   
2        3                            Grumpier Old Men (1995)   
3        4                           Waiting to Exhale (1995)   
4        5                 Father of the Bride Part II (1995)   
5        6                                        Heat (1995)   
6        7                                     Sabrina (1995)   
7        8                                Tom and Huck (1995)   
8        9                                Sudden Death (1995)   
9       10                                   GoldenEye (1995)   
10      11                     American President, The (1995)   
11      12                 Dracula: Dead and Loving It (1995)   
12      13                                       Balto (1995)   
13      14                                       Nixon (1995)   
14      15               

In [4]:
users = pd.read_csv('users.dat',sep='::',header=None,engine='python',encoding = 'latin-1')
print(users)

         0  1   2   3      4
0        1  F   1  10  48067
1        2  M  56  16  70072
2        3  M  25  15  55117
3        4  M  45   7  02460
4        5  M  25  20  55455
5        6  F  50   9  55117
6        7  M  35   1  06810
7        8  M  25  12  11413
8        9  M  25  17  61614
9       10  F  35   1  95370
10      11  F  25   1  04093
11      12  M  25  12  32793
12      13  M  45   1  93304
13      14  M  35   0  60126
14      15  M  25   7  22903
15      16  F  35   0  20670
16      17  M  50   1  95350
17      18  F  18   3  95825
18      19  M   1  10  48073
19      20  M  25  14  55113
20      21  M  18  16  99353
21      22  M  18  15  53706
22      23  M  35   0  90049
23      24  F  25   7  10023
24      25  M  18   4  01609
25      26  M  25   7  23112
26      27  M  25  11  19130
27      28  F  25   1  14607
28      29  M  35   7  33407
29      30  F  35   7  19143
...    ... ..  ..  ..    ...
6010  6011  M  35  15  80538
6011  6012  M  35  15  02871
6012  6013  F 

In [5]:
ratings = pd.read_csv('ratings.dat',sep='::',header=None,engine='python',encoding = 'latin-1')
print(ratings)

            0     1  2          3
0           1  1193  5  978300760
1           1   661  3  978302109
2           1   914  3  978301968
3           1  3408  4  978300275
4           1  2355  5  978824291
5           1  1197  3  978302268
6           1  1287  5  978302039
7           1  2804  5  978300719
8           1   594  4  978302268
9           1   919  4  978301368
10          1   595  5  978824268
11          1   938  4  978301752
12          1  2398  4  978302281
13          1  2918  4  978302124
14          1  1035  5  978301753
15          1  2791  4  978302188
16          1  2687  3  978824268
17          1  2018  4  978301777
18          1  3105  5  978301713
19          1  2797  4  978302039
20          1  2321  3  978302205
21          1   720  3  978300760
22          1  1270  5  978300055
23          1   527  5  978824195
24          1  2340  3  978300103
25          1    48  5  978824351
26          1  1097  4  978301953
27          1  1721  4  978300055
28          1 

In [6]:
#Preparing the training set and the test set
tr_set= pd.read_csv('u1.base',delimiter='\t')
print(tr_set)

         1   1.1  5  874965758
0        1     2  3  876893171
1        1     3  4  878542960
2        1     4  3  876893119
3        1     5  3  889751712
4        1     7  4  875071561
5        1     8  1  875072484
6        1     9  5  878543541
7        1    11  2  875072262
8        1    13  5  875071805
9        1    15  5  875071608
10       1    16  5  878543541
11       1    18  4  887432020
12       1    19  5  875071515
13       1    21  1  878542772
14       1    22  4  875072404
15       1    25  4  875071805
16       1    26  3  875072442
17       1    28  4  875072173
18       1    29  1  878542869
19       1    30  3  878542515
20       1    32  5  888732909
21       1    34  2  878542869
22       1    35  1  878542420
23       1    37  2  878543030
24       1    38  3  878543075
25       1    40  3  876893230
26       1    41  2  876892818
27       1    42  5  876892425
28       1    43  4  878542869
29       1    45  5  875241687
...    ...   ... ..        ...
79969  9

In [7]:
tr_set=np.array(tr_set,dtype='int')
print(tr_set)

[[        1         2         3 876893171]
 [        1         3         4 878542960]
 [        1         4         3 876893119]
 ...
 [      943      1188         3 888640250]
 [      943      1228         3 888640275]
 [      943      1330         3 888692465]]


In [8]:
te_set= pd.read_csv('u1.test',delimiter='\t')
te_set=np.array(tr_set,dtype='int')
print(te_set)

[[        1         2         3 876893171]
 [        1         3         4 878542960]
 [        1         4         3 876893119]
 ...
 [      943      1188         3 888640250]
 [      943      1228         3 888640275]
 [      943      1330         3 888692465]]


In [9]:
nb_users=int(max(max(tr_set[:,0]),max(te_set[:,0])))
nb_movies=int(max(max(tr_set[:,1]),max(te_set[:,1])))

In [10]:
#Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data=[]
    for id_users in range(1,nb_users+1):
        id_movies=data[:,1][data[:,0]==id_users]
        id_ratings=data[:,2][data[:,0]==id_users]
        ratings=np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))
    return new_data
tr_set=convert(tr_set)
te_set=convert(te_set)

In [11]:
#Converting the data into torch sensors
tr_set = torch.FloatTensor(tr_set)
te_set=torch.FloatTensor(te_set)

In [12]:
#Converting the ratings into binary ratings 1 (liked) or 0 (liked)
tr_set[tr_set==0] = -1
tr_set[tr_set==1] = 0
tr_set[tr_set==2] = 0
tr_set[tr_set>=3] = 1
te_set[te_set==0] = -1
te_set[te_set==1] = 0
te_set[te_set==2] = 0
te_set[te_set>=3] = 1

In [13]:
#Creating the architecture of neural network
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh,nv)
        self.a=torch.randn(1, nh) #prob of hidden node given visible node
        self.b=torch.randn(1,nv) #prob of visible node given hidden node
    def sample_h(self,x):
        wx= torch.mm(x,self.W.t())
        activation = wx + self.a.expand_as(wx)#activate the hidden nodes
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self,y):
        wy= torch.mm(y,self.W)
        activation = wy + self.b.expand_as(wy)#activate the hidden nodes
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(),ph0)- torch.mm(vk.t(),phk)
        self.b += torch.sum((v0-vk), 0)
        self.a += torch.sum((ph0-phk), 0)

In [14]:
nv = len(tr_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv,nh)

In [17]:
#Training the RBM
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s= 0.0
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = tr_set[id_user:id_user+batch_size]
        v0 = tr_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        #now k steps for contrastive divergence
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch : '+str(epoch)+' loss: '+str(train_loss/s))

RuntimeError: The size of tensor a (1682) must match the size of tensor b (100) at non-singleton dimension 1

In [ ]:
#Testing the RBM
test_loss = 0
s= 0.0
for id_user in range(nb_users):
    v = tr_set[id_user:id_user+1]#training set used to activate hidden neurons for test set
    vt = te_set[id_user:id_user+1]
    #for k in range(1):because this is not random walk but blind walk
    if len(vt[vt>=0])>0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(train_loss/s))